# Seismic detection using neural network with CNN and LSTM layers combined

## Imports

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout, Input, LSTM, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

## Processing functions

In [2]:
# Load seismic data from the CSV file
def load_seismic_data(file_path):
    data = pd.read_csv(file_path)
    return data

# Preprocess the velocity data
def preprocess_data(velocity_data):
    # Normalize the velocity data (zero mean, unit variance)
    normalized_data = (velocity_data - np.mean(velocity_data)) / np.std(velocity_data)
    
    return normalized_data

def calculate_derivative(velocity_data):
    # Get the derivative feature at each time step
    derivative = np.diff(velocity_data, prepend=velocity_data[0])  # Calculate the derivative
    return derivative

def preprocess_derivative(derivative_data):
    # Normalize the derivative values
    normalized_derivative = (derivative_data - np.mean(derivative_data)) / np.std(derivative_data)
    return normalized_derivative


## Model architecture

In [3]:
# Model architecture: 1D Convolutional Neural Network
def build_model_with_lstm(input_shape):
    model = Sequential()

   # Input layer
    model.add(Input(shape=input_shape))
    
    # First Conv1D layer
    model.add(Conv1D(64, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    
    # Second Conv1D layer
    model.add(Conv1D(128, kernel_size=5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    # Using LSTM for sequence learning
    model.add(Bidirectional(LSTM(128, return_sequences=False)))

    # Dense and output layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5)) # Dropout to prevent overfitting
    model.add(Dense(1))  # Output the predicted (normalized) index
    
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    
    return model


## Load and process data for training

In [4]:
# Load and process the catalog
cat_directory = './data/lunar/training/catalogs/'
cat_file = cat_directory + 'apollo12_catalog_GradeA_final.csv'
cat = pd.read_csv(cat_file)

# Prepare training data (seismic data + labels for quake start detection)
X_train = []
y_train = []

# Define a max length for padding/truncating
MAX_TIMESTEPS = 60000  # the maximum expected recording length

# Loop through the catalog
for i, (file_name, _, start_time, _, quake_type) in cat.iterrows():
    file_path = f"./data/lunar/training/data/S12_GradeA/{file_name}.csv"
    if not os.path.isfile(file_path):
        continue

    # Load seismic data
    data_chunk = load_seismic_data(file_path)
    velocity = data_chunk['velocity(m/s)'].values
    time = data_chunk['time_rel(sec)'].values

    # Preprocess the data
    velocity_derivative = calculate_derivative(velocity)
    processed_velocity = preprocess_data(velocity)
    processed_derivative = preprocess_derivative(velocity_derivative)

    # Stack both velocity and its derivative as input features
    stacked_data = np.stack((processed_velocity, processed_derivative), axis=-1)  # Shape: (timesteps, 2 features)

    # Labeling: Create a label with the exact time index of the quake start
    start_index = np.argmin(np.abs(time - start_time))  # Closest index to the start time
    
    # Append processed data and the start index to the training set
    X_train.append(stacked_data)
    y_train.append(start_index)  # The label is the index of the quake start

# Convert lists to numpy arrays with consistent time series length using padding
X_train_padded = pad_sequences(X_train, maxlen=MAX_TIMESTEPS, dtype='float32', padding='post', truncating='post')

# Convert lists to numpy arrays for model training
X_train = np.array(X_train_padded)
y_train = np.array(y_train)  # The labels are now the start indices

# Check the shape of X_train and y_train
print("X_train shape:", X_train.shape)  # Should be (samples, timesteps, 1)
print("y_train shape:", y_train.shape)  # Should be (samples,)

# Split data into train and test sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#normalize to 0-1
y_train = y_train / MAX_TIMESTEPS  # MAX_TIMESTEPS is 60000 or the max length of the sequence
y_val = y_val / MAX_TIMESTEPS

# Build the model
input_shape = (X_train.shape[1], X_train.shape[2])
model = build_model_with_lstm(input_shape)

X_train shape: (75, 60000, 2)
y_train shape: (75,)


2024-10-06 10:34:37.569732: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-10-06 10:34:37.569782: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-10-06 10:34:37.569789: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-10-06 10:34:37.569819: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-10-06 10:34:37.569841: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Model training

In [9]:
# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_val, y_val),verbose=2)

Epoch 1/30
2/2 - 7s - 3s/step - loss: 8.1913 - mae: 2.5096 - val_loss: 4.7614 - val_mae: 1.8573
Epoch 2/30
2/2 - 6s - 3s/step - loss: 8.3567 - mae: 2.5087 - val_loss: 4.7222 - val_mae: 1.8626
Epoch 3/30
2/2 - 6s - 3s/step - loss: 8.0280 - mae: 2.5400 - val_loss: 4.6695 - val_mae: 1.8693
Epoch 4/30
2/2 - 6s - 3s/step - loss: 8.9551 - mae: 2.6352 - val_loss: 4.6282 - val_mae: 1.8675
Epoch 5/30
2/2 - 6s - 3s/step - loss: 7.7815 - mae: 2.4365 - val_loss: 4.5892 - val_mae: 1.8522
Epoch 6/30
2/2 - 7s - 3s/step - loss: 7.2905 - mae: 2.3453 - val_loss: 4.5430 - val_mae: 1.8357
Epoch 7/30
2/2 - 6s - 3s/step - loss: 7.9147 - mae: 2.5103 - val_loss: 4.5122 - val_mae: 1.8389
Epoch 8/30
2/2 - 6s - 3s/step - loss: 7.1787 - mae: 2.3237 - val_loss: 4.5237 - val_mae: 1.8716
Epoch 9/30
2/2 - 6s - 3s/step - loss: 7.6946 - mae: 2.3877 - val_loss: 4.6423 - val_mae: 1.9114
Epoch 10/30
2/2 - 6s - 3s/step - loss: 7.0228 - mae: 2.2793 - val_loss: 4.7810 - val_mae: 1.9376
Epoch 11/30
2/2 - 6s - 3s/step - loss: 

## Model evaluation

In [10]:
# Evaluate the model
loss, mae = model.evaluate(X_val, y_val)
print(f"Validation MAE: {mae:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 4.6932 - mae: 1.8228
Validation MAE: 1.8228


## Predictions

The code outputs prediction to both CSV files (that can be used later to score the model), and images, which provides the ability to visually assess the model

In [11]:
# Define the directory for test files
test_directory = "./data/lunar/test/data/S12_GradeB/"
output_plot_dir = "./predictions_plots_v2/"  # Directory to save plots
output_csv_file = "./predictions_output_v2.csv"  # CSV file to save the predictions

# Ensure output directory exists
os.makedirs(output_plot_dir, exist_ok=True)

# Function to predict moonquake start, save plot, and store results in a CSV
def predict_and_plot_for_all_files(model, test_directory, max_timesteps=60000):
    predictions = []  # Store the results here
    
    # Open CSV file for writing
    with open(output_csv_file, mode='w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        # Write the header for the CSV file
        csv_writer.writerow(["filename", "time_rel(sec)"]) # Write the column names
        
        # Iterate through all the CSV test files in the test directory
        for file_name in os.listdir(test_directory):
            if file_name.endswith(".csv"):  # Only process CSV files
                file_path = os.path.join(test_directory, file_name)
                print(f"Processing file: {file_name}")  # For debugging/logging
                
                # Load seismic data from the file
                data_chunk = load_seismic_data(file_path)
                velocity = data_chunk['velocity(m/s)'].values
                time = data_chunk['time_rel(sec)'].values
                
                velocity_derivative = calculate_derivative(velocity)
                processed_velocity = preprocess_data(velocity)
                processed_derivative = preprocess_derivative(velocity_derivative)

                # Stack both velocity and its derivative as input features
                stacked_data = np.stack((processed_velocity, processed_derivative), axis=-1)  # Shape: (timesteps, 2 features)
                
                # Preprocess the velocity data
                stacked_data = pad_sequences([stacked_data], maxlen=max_timesteps, dtype='float32', padding='post', truncating='post')

                # Make prediction
                predicted_index = model.predict(stacked_data)
                predicted_index = int(predicted_index[0] * MAX_TIMESTEPS)  # Scale back to original range
                
                # Get the predicted start time using the index
                predicted_start_time = time[predicted_index]

                # Plotting the seismic data and the predicted moonquake start
                plt.figure(figsize=(10, 6))
                plt.plot(time, velocity, label="Seismic Velocity Data", color='blue')
                
                # Add a red line for the predicted quake start time
                plt.axvline(x=predicted_start_time, color='red', linestyle='--', label=f"Predicted Quake Start: {predicted_start_time:.2f}s")
                
                plt.title(f"Moonquake Prediction for {file_name}")
                plt.xlabel("Time (seconds)")
                plt.ylabel("Velocity (m/s)")
                plt.legend()
                
                # Save the plot to a file
                plot_filename = os.path.join(output_plot_dir, f"{file_name}_prediction_plot.png")
                plt.savefig(plot_filename)
                plt.close()  # Close the plot to avoid displaying it during processing

                # Store the result (file_name, predicted_start_time) in the CSV
                csv_writer.writerow([file_name, predicted_start_time])

                # Append the result to the predictions list for further use if needed
                predictions.append((file_name, predicted_start_time))
    
    return predictions

# Example usage
predictions = predict_and_plot_for_all_files(model, test_directory)

# Output the predictions
for file_name, predicted_start_time in predictions:
    print(f"File: {file_name}, Predicted Moonquake Start Time: {predicted_start_time}")

Processing file: xa.s12.00.mhz.1970-05-23HR00_evid00027.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
Processing file: xa.s12.00.mhz.1977-04-11HR00_evid00915.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
Processing file: xa.s12.00.mhz.1970-07-18HR00_evid00036.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
Processing file: xa.s12.00.mhz.1971-11-24HR00_evid00156.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
Processing file: xa.s12.00.mhz.1972-12-06HR00_evid00342.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
Processing file: xa.s12.00.mhz.1974-03-14HR00_evid00506.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
Processing file: xa.s12.00.mhz.1973-11-22HR00_evid00475.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
Processing file: xa.s12.00.mhz.1971-04-08HR01_evid00083.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
Processing file: xa.s12.00.mhz.1973-10-10HR00_evid00463.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
Processing file: xa.s12.00.mhz.1974-07-25HR05_evid00553.csv
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
Processing